In [3]:
import numpy as np
from sequentialGP import *
from UQpy.surrogates import *
from UQpy.utilities.MinimizeOptimizer import MinimizeOptimizer
from UQpy.utilities import Matern
import chaospy

Load the data, consisting of 
- input parameter matrix $x \in \mathbb{R}^{n_{\text{train}} \times 2}$ with Lennard Jones parameters
- output matrix $y \in \mathbb{R}^{n_{\text{train}} \times 2}$ with $2$ properties ($P_\text{SIM}$ in MPa at $326,16$ K and $408,34$ K) over the training data set
- measurement data $z \in \mathbb{R}^{2}$

In [24]:
x = np.array([
        [152.5953, 3.5638],
        [144.9884, 3.4765],
        [135.5627, 3.5049],
        [155.4567, 3.446],
        [149.7889, 3.4676],
        [146.5005, 3.5388],
        [129.3503, 3.5482],
        [137.9998, 3.5576]
    ])

y = np.array([
        [1.2065000e-01, 1.0170000e+00],
        [1.7699000e-01, 1.3480000e+00],
        [2.4444000e-01, 1.6603000e+00],
        [1.2370000e-01, 1.0272000e+00],
        [1.5692000e-01, 1.2062000e+00],
        [1.5804000e-01, 1.2238000e+00],
        [2.8667000e-01, 1.9978000e+00],
        [2.0888000e-01, 1.5246000e+00]
    ])

z = [0.1205, 0.97558]
num_properties = len(z)

The next cell: 
- defines the bounds for the Lennard Jones parameters $(\varepsilon,\sigma)$
- defines the joint distribution over the parameter domain (uniform and independent)
- sets the uncertainty of the measurement data

In [25]:
np.random.seed(5)

# bounds
eps_range = [130.0, 160.0]
sig_range = [3.4, 3.6]
bounds = [(130,160),(3.4,3.6)]
domain = np.array([eps_range,sig_range])

# distribution
joint_X = chaospy.J(chaospy.Uniform(eps_range[0],eps_range[1]), chaospy.Uniform(sig_range[0],sig_range[1]))

# measurement noise
percentageUncertainty = np.array([0.3, 0.3]) 
sigma_noise = np.multiply(percentageUncertainty/100,z)

# Samples in outer loop Monte Carlo approximation
n_samples_X = 20

Initialize the Gaussian Process (GP) surrogate model with the training data and initialize object for sequential design:

In [26]:
# GP regression on initial training data
optimizer = MinimizeOptimizer(method='slsqp')
K = [GaussianProcessRegression(regression_model=ConstantRegression(), kernel=Matern(1.5),
                                optimizer=optimizer, optimizations_number=5, hyperparameters=[100,1,0.5],
                                random_state=2, noise=False, normalize=True) for i in range(num_properties)]

sGP = SequentialGP(K,x,y,joint_X)

Finally, we compute the Bayes risk over a grid. Note that the points could also be determined randomly or an optimization routine could be used as well. A grid is simple for illustration and plotting.

In [27]:
n_samples_risk = 15  
eps_candidates = np.linspace(bounds[0][0],bounds[0][1],n_samples_risk)
sigma_candidates = np.linspace(bounds[1][0],bounds[1][1],n_samples_risk)
[eps_mg,sig_mg] = np.meshgrid(eps_candidates,sigma_candidates)

r_mg = np.zeros((n_samples_risk,n_samples_risk))

for ind1 in range(n_samples_risk):
    for ind2 in range(n_samples_risk):
        r_mg[ind1,ind2] = sGP.Bayes_risk(np.array([eps_mg[ind1,ind2],sig_mg[ind1,ind2]]),z,sigma_noise,n_samples_X)

# find minimum on grid
ind_new = np.argmin(r_mg)
ind_new_2d = np.unravel_index(ind_new, r_mg.shape)
eps_new = eps_mg[ind_new_2d]
sig_new = sig_mg[ind_new_2d]

print(eps_new,sig_new)

157.85714285714286 3.5428571428571427
